In [ ]:
import os
import warnings
import pandas as pd
import requests
from bs4 import BeautifulSoup
import wget
import ssl
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

data_atual = datetime.now()
data_atual_f = data_atual.date().strftime('%d-%m-%Y')
data_ontem = data_atual - timedelta(days=1)
data_ontem = data_ontem.date().strftime('%d-%m-%Y')

ssl._create_default_https_context = ssl._create_unverified_context

url = f'https://extrator.sefaz.al.gov.br/DESPESAS/COMPARATIVO-DESPESAS/despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_ontem}.xlsx'
arquivo = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python', f'despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_ontem}.xlsx')
arquivo_csv = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python', f'despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_ontem}.csv')
base_arquivo = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/SCRIPTS/Bases empenhado - Layne', f'Base Acompanhamento - atualizado {data_atual_f}.xlsx')

# Baixar o arquivo
wget.download(url, arquivo)
df = pd.read_excel(arquivo)

df_f = df[['DESCRICAO_UO', 'PT', 'PT_DESCRICAO', 'NATUREZA6', 'VALOR_EMPENHADO', 'MES']]
for i in ['DESCRICAO_UO', 'PT', 'PT_DESCRICAO', 'NATUREZA6', 'MES']:
    df_f[i] = df_f[i].astype(str)

# concatenando as colunas para o merge
df_x = df_f
df_x['concat'] = df_x['DESCRICAO_UO']
for i in ['PT', 'NATUREZA6']:
    df_x['concat'] = df_x['concat'] + df_x[i]

# base com as informacoes
info = pd.read_excel('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python/Base acompanhamento Layne.xlsx')

# fazendo o merge para cada valor e cada mês

for i in df_f['MES'].unique():
    info[i] = ''
    teste = df_x.loc[df_x['MES'] == i]
    for j in info['concat'].values:
     info.loc[info.concat == j, i] = teste.loc[teste.concat == j]['VALOR_EMPENHADO'].sum()

# Dropando o concat

info.drop(columns='concat', inplace=True)

# Padronizando as colunas e organiznado

info.rename(columns={'DESCRICAO_UO': 'UO', 'PT': 'PROGRAMA DE TRABALHO',
                      'PT_DESCRICAO': 'PROGRAMA DE TRABALHO DESCRIÇÃO',
                        'NATUREZA6': 'NATUREZA'}, inplace=True)

# Padronizando os meses

meses_padronizados = {
    '1': 'JAN', '2': 'FEV', '3': 'MAR', '4': 'ABR',
    '5': 'MAI', '6': 'JUN', '7': 'JUL', '8': 'AGO',
    '9': 'SET', '10': 'OUT', '11': 'NOV', '12': 'DEZ'
}

info.rename(columns=meses_padronizados, inplace=True)

# Padronizando os PTs

pt_padronizados = {'10302101550690000': '10.302. 1015. 5069',
                   '6122000420010000': '06.122. 0004. 2001',
                   '20605104335800000': '20.605. 1043. 3580',
                   '8306104352110000': '08.306. 1043. 5211',
                   '12368101237620000': '12.368. 1012. 3762',
                   '12368101237620000': '12.368. 1012. 3762',
                   '6181101738370000': '06.181. 1017. 3837',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14421000451850000': '14.421. 0004. 5185',
                   '14421000451850000': '14.421. 0004. 5185',
                   '14421000451850000': '14.421. 0004. 5185',
                   '6122101952750000': '06.122. 1019. 5275',
                   '6122101952750000': '06.122. 1019. 5275'}

info['PROGRAMA DE TRABALHO'] = info['PROGRAMA DE TRABALHO'].astype(str)
info['PROGRAMA DE TRABALHO'] = info['PROGRAMA DE TRABALHO'].map(pt_padronizados)


info.to_excel(base_arquivo, index=False)
info